In [19]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris,load_wine
from dsgd import DSClassifierMultiQ

In [91]:
iris = load_iris()
X = iris.data
y = iris.target
np.random.seed(42)
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
y = y[indices]
X_train = X[:100]
y_train = y[:100]
X_test = X[100:]
y_test = y[100:]

In [95]:
DSC = DSClassifierMultiQ(3, max_iter=150, debug_mode=True, lossfn="MSE", min_dloss=0.0001, lr=0.005, precompute_rules=True)
losses, epoch, dt = DSC.fit(X_train, y_train, add_single_rules=True, single_rules_breaks=3, add_mult_rules=False, column_names=iris.feature_names, print_every_epochs=1, print_final_model=False)

Optimization started
Processing epoch	150	0.0273	
Training time: 15.54s, epochs: 150

Least training loss reached: 0.027


In [ ]:
DSC.print_most_important_rules(threshold=0.5)



Most important rules for class 0

	[0.574] R1: 5.239 < sepal length (cm) < 5.798
			0: 0.413	1: 0.373	2: 0.012	Unc: 0.202

	[0.537] R8: petal length (cm) < 2.425
			0: 0.537	1: 0.000	2: 0.000	Unc: 0.463

	[0.523] R12: petal width (cm) < 0.628
			0: 0.523	1: 0.000	2: 0.000	Unc: 0.477

Most important rules for class 1

	[0.643] R9: 2.425 < petal length (cm) < 3.635
			0: 0.000	1: 0.643	2: 0.000	Unc: 0.357

	[0.636] R10: 3.635 < petal length (cm) < 4.845
			0: 0.000	1: 0.636	2: 0.000	Unc: 0.364

	[0.616] R4: sepal width (cm) < 2.762
			0: 0.000	1: 0.423	2: 0.473	Unc: 0.104

	[0.576] R14: 1.144 < petal width (cm) < 1.660
			0: 0.000	1: 0.576	2: 0.000	Unc: 0.424

	[0.546] R1: 5.239 < sepal length (cm) < 5.798
			0: 0.413	1: 0.373	2: 0.012	Unc: 0.202

	[0.545] R2: 5.798 < sepal length (cm) < 6.357
			0: 0.000	1: 0.344	2: 0.520	Unc: 0.136

	[0.539] R13: 0.628 < petal width (cm) < 1.144
			0: 0.000	1: 0.539	2: 0.000	Unc: 0.461

Most important rules for class 2

	[0.670] R2: 5.798 < sepal len

In [96]:
#predecir 2 valores
X_val = np.array([[5.0, 3.6, 1.4, 0.2], [6.0, 1.6, 2.4, np.nan]])
y_val = DSC.predict(X_val, one_hot=False)
print("Predicción de los valores: ", y_val)

Predicción de los valores:  [0 2]


In [97]:
import torch
DSC.model.eval()
DSC.model.clear_rmap()

if "values" in dir(X_val):
    X_val = X_val.values
X_val = np.insert(X_val, 0, values=np.arange(0, len(X_val)), axis=1)
with torch.no_grad():
    Xt = torch.Tensor(X_val).to(DSC.device)
    y_val = DSC.model(Xt).numpy()
    #diferencia con como se predice con defecto
    for r in DSC.model.rmap:
        print(f"Valor del punto ({X_val[r][1:]}): ", y_val[r])
        print("Regla: ", DSC.model.rmap[r])
        preds = [DSC.model.preds[i] for i in DSC.model.rmap[r]]
        params = [DSC.model._params[i] for i in DSC.model.rmap[r]]
        for i in range(len(preds)):
            print("Regla: ", preds[i].caption)
            print("Pesos: ", params[i].detach().numpy())
        """ data = []
        for i in range(len(preds)):
            param_values = params[i].detach().numpy().tolist()
            row = {"Reglas": preds[i].caption}
            for j, val in enumerate(param_values):
                row[f"Parametro_{j}"] = val
            data.append(row)
        df = pd.DataFrame(data)
        print(df) """


Valor del punto ([5.  3.6 1.4 0.2]):  [0.8973639  0.05131807 0.05131807]
Regla:  [0, 7, 8, 12]
Regla:  sepal length (cm) < 5.239
Pesos:  [0.46178505 0.         0.         0.5382149 ]
Regla:  sepal width (cm) > 3.386
Pesos:  [0.46807376 0.         0.         0.5319263 ]
Regla:  petal length (cm) < 2.425
Pesos:  [0.52823305 0.         0.         0.47176692]
Regla:  petal width (cm) < 0.628
Pesos:  [0.57658416 0.         0.         0.4234158 ]
Valor del punto ([6.  1.6 2.4 nan]):  [0.03512089 0.41067928 0.5541998 ]
Regla:  [2, 4, 8]
Regla:  5.798 < sepal length (cm) < 6.357
Pesos:  [0.         0.36198312 0.5162954  0.12172149]
Regla:  sepal width (cm) < 2.762
Pesos:  [0.         0.45076907 0.463162   0.08606894]
Regla:  petal length (cm) < 2.425
Pesos:  [0.52823305 0.         0.         0.47176692]


In [8]:
print("Columnas: ", iris.feature_names)

Columnas:  ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [94]:
from dsgd import DSRule
DSC = DSClassifierMultiQ(3, max_iter=150, debug_mode=True, lossfn="MSE", min_dloss=0.0001, lr=0.005, precompute_rules=True)
DSC.model.add_rule(DSRule(lambda x: x[0] > 1, "Regla de ejemplo con sepal length (cm) mayor a 1"))
DSC.model.add_rule(DSRule(lambda x: x[0] < 1, "Regla de ejemplo con sepal length (cm) menor a 1"))
DSC.model.add_rule(DSRule(lambda x: x[1] > 1, "Regla de ejemplo con sepal width (cm) mayor a 1"))
DSC.model.add_rule(DSRule(lambda x: x[1] < 1, "Regla de ejemplo con sepal width (cm) menor a 1"))
DSC.model.add_rule(DSRule(lambda x: x[2] > 1, "Regla de ejemplo con petal length (cm) mayor a 1"))
DSC.model.add_rule(DSRule(lambda x: x[2] < 1, "Regla de ejemplo con petal length (cm) menor a 1"))
DSC.model.add_rule(DSRule(lambda x: x[3] > 1, "Regla de ejemplo con petal width (cm) mayor a 1"))
DSC.model.add_rule(DSRule(lambda x: x[3] < 1, "Regla de ejemplo con petal width (cm) menor a 1"))
losses, epoch, dt = DSC.fit(X_train, y_train, add_single_rules=False, add_mult_rules=False, column_names=iris.feature_names, print_every_epochs=1, print_final_model=False)
DSC.print_most_important_rules(threshold=0.5)

Optimization started
Processing epoch	118	0.1219	
Training time: 13.22s, epochs: 118

Least training loss reached: 0.122


Most important rules for class 0

	[0.546] R7: Regla de ejemplo con petal width (cm) menor a 1
			0: 0.546	1: 0.000	2: 0.000	Unc: 0.454

	[0.502] R0: Regla de ejemplo con sepal length (cm) mayor a 1
			0: 0.502	1: 0.000	2: 0.000	Unc: 0.498

Most important rules for class 1

	[0.684] R6: Regla de ejemplo con petal width (cm) mayor a 1
			0: 0.000	1: 0.468	2: 0.532	Unc: 0.000

Most important rules for class 2

	[0.729] R6: Regla de ejemplo con petal width (cm) mayor a 1
			0: 0.000	1: 0.468	2: 0.532	Unc: 0.000
